# 소프트웨어 컴포넌트
대규모 시스템을 확장해야한다고 할 때 대규모 시스템이 어떠한 컴포넌트로 구성되어 있는 지를 명심해야 함  
시스템은 파이썬 패키지를 만드는 간단한 것에서부터 마이크로서비스 아키텍처의 복잡한 시나리오에 이르기까지 다양한 단계를 가질 수 있음  

대규모에서 잘 동작하기 위한 유일한 방법은 모든 부분이 인터페이스, 즉 계약에 동의하는 것임  

## 패키지
파이썬 패키지는 소프트웨어를 배포하고 보다 일반적인 방식으로 코드를 재사용하기 위한 편리한 방법임  
빌드 패키지는 아티펙트(artifact) 저장소에 업로드 할 수 있으며 다른 아티펙트 저장소에서 이 패키지를 다운로드 할 수 있음  
이렇게 함으로써 코드의 재사용성을 높이고 개념적인 무결성을 얻을 수 있음  

**저장소에 업로드 가능한 형태로 파이썬 프로젝트를 패키징하는 기본적인 방법에 대해 알아보기**  
오픈 소스 라이브러리 외에도 일부 특수 기능이 필요하여 자체적으로 작은 라이브러리를 만들었다고 가정  
이러한 작업을 보다 효율화하려면 추상화를 라이브러리 형태로 구현하고, 팀원들이 해당 라이브러리에서 제공하는 관용구를 사용하도록 독려해야함  
그렇게 하면 버그를 줄이고 실수를 피할 수 있기 때문

**비슷한 시나리오  **
> 애플리케이션에서 .tar.gz 파일과 같은 특정한 포맷의 파일에서 데이터를 추출할 때 경로 통과 공격을 하는 악성 파일로 인해 보안 문제가 발생할 수 있음  
이를 해결하기 위해 기본 기능을 추상화하여 추가 보안 검사를 하는 래핑 함수를 라이브러리에 만들 수 있음  

> 특정 형식으로 작성되거나 분석되어야하는 설정 파일이 있을 때 헬퍼 함수를 만들고 모든 프로젝트에서 사용하도록 하기

이는 DRY원칙(코드 중복 방지, 재사용 권장)을 준수할 뿐만 아니라 추상화된 기능이 어떻게 사용되어야 하는지에 대한 단일 참조점을 나타내므로 개념적 무결설 달성에 도움이 됨  

일반적인 라이브러리 기본 구조  
```bash
├── Makefile
├── README.rst
├── setup.py
├── src
│   ├── apptool
│   ├── common.py
│   ├── __init__.py
│   └── parse.py
└── tests
     ├── integration
     └── unit
``` 

패키지 정의가 들어있는 **setup.py** 파일이 가장 중요한 파일임  
- 이 파일에는 프로젝트의 모든 중요한 정의(요구사항, 의존성, 이름, 설명 등)가 지정되어 있음    

src 밑에 있는 **apptool 디렉토리**는 우리가 작업하고 있는 라이브러리의 이름  
- 일반적으로 파이썬 프로젝트에서 필요한 모든 것을 이곳에 배치함  

**src.py 파일의 예**
```python
from setuptools import find_packages, setup  

with open("README.rst", "r") as longdesc:
    long_description = longdesc.read()
    
setup(
    name="apptool", 
    description="패키지 설명",
    long_description=long_description,
    author="저자이름",
    version="0.1.0",
    packages=find_packages(where="src/"),
    package_dir={"": "src"},
)
```

* setup 함수의 name 인자
    - 저장소에서 사용할 패키지의 이름
    - 이 이름을 사용해 설치 명령 가능 (이 경우에는 pip install apptool)
    - 이 이름을 염격하게 프로젝트 디렉토리(src/apptool)와 일치할 필요는 없지만 이왕이면 일치시키는 것이 좋음  
        - 두 이름이 일치하면 pip install apptool로 설치하고 코드에서 from apptool import myutil 형식으로 사용 가능
        - 실제로 실행할 땐 설치시에는 setup.py에서 지정한 name을 참조하고, 코드에서는 src/ 디렉토리를 참조
* 버전은 배포에 중요한 정보가 되며 패키지를 지정하는 데 사용  
    - find_packages() 함수를 사용하면 자동으로 모든 패키지를 검색함  
    - 이 경우에는 src/ 디렉토리에서 검색함  
    - 이 디렉토리에서 파일을 검색하면 프로젝트의 범위가 아닌 파일과 섞이는 것을 피할 수 있음  
    
패키지는 다음 명령어를 통해 빌드할 수 있음  
```shell
$VIRTUAL_ENV/bin/pip install -U setuptools wheel
$VIRTAUL_ENV/bin/python setup.py sdist bdist_wheel
```
이 명령어를 실행하면 산출물을 dist/ 디렉토리에 만들고 이것을 나중에 PyPi나 회사의 내부 패키지 저장소에 업로드할 수 있음  

파이썬 프로젝트 패키징의 핵심  
- 플랫폼에 독립적이며 로컬 설치에 의존하지 않았는지 테스트하고 검증해야함
    - 소스 파일을 src/ 디렉토리에 놓음으로써 보다 수월하게 할 수 있음  
- 단위 테스트를 패키지에 같이 배포하지 않음 
- 의존성 분리 -> 프로젝트가 필요로 하는 것과 개발자가 필요로 하는 것은 다름  
- 가장 많이 요구되는 명령에 대해서는 진입점을 만드는 것이 좋음  

setup.py 파일은 다른 여러 개의 파라미터나 설정을 지원하며 훨씬 더 복잡한 방식으로 조절할 수 있음  
패키지에 여러 운영체제 라이브러리가 설치되어야하는 경우 setup.py 파일에 적당한 논리를 작성하여 필요한 컴파일을 하고 빌드하는 것이 좋음  
이렇게 하면 무언가가 잘못도니 겅우 설치 프로세스 초기에 빨리 오류를 발생시켜 사용자가 확인하게 할 수 있으며, 사용자는 의존성을 보다 빨리 확인하고 수정하여 계속 진행할 수 있음  

의존성을 설치하는 것은 플랫폼에 상관 없이 모든 개발자가 쉽게 사용하는 데 있어서 걸림돌이 될 수 있음  
이 장애물을 극복하는 가장 좋은 방법은 도커(Docker) 이미지를 만들어 플랫폼을 추상화하는 것임  

## 컨테이너  
여기에서 컨테이너는 파이썬 컨테이너(&#95;&#95;contains&#95;&#95; 메서드를 가진 객체)와는 다른 의미의 컨테이너임  
컨테이너는 특정한 제약사항을 가지고 격리된 상태로 실행되는 프로세스  
=> 도커 컨테이너를 의미

컨테이너는 또 다른 형태의 소프트웨어를 전달하는 수단  
코드를 재사용하고 특정 논리가 모여있는 단일 장소를 사용하는 라이브러리나 프레임워크를 목표로 한다면 이전 섹션에서 살펴본 방식으로 파이썬 패키지를 만드는 것이 더 적합함  

컨테이너의 겨우 라이브러리가 아닌 애플리케이션을 생성할 때 사용함  
그러나 애플리케이션이나 플랫폼이 반드시 전체 서비스를 의미하는 것은 아님  
컨테이너를 만드는 이유는 작고 명확한 서비스를 나타내는 작은 컴포넌트를 만들기 위함임  

도커 컨테이너는 실행할 이미지가 필요하며 이 이미지는 다른 기본(base) 이미지에서 만들어짐  
생성하는 이미지 역시 다른 컨테이너의 기본 이미지로 사용될 수 있음  
이 기반 컨테이너 위에 운영체제에서 필요한 것들을 포함해 의존성을 가진 패키지 등 모든 것들을 설치할 수 있음  

컨테이너는 애플리케이션이 표준적인 실행방법을 갖도록 도와줄 뿐만 아니라 개발 프로세스를 수월하게 만듦  
-> 환경에 따른 시나리오 재현, 테스트 복제, 새로운 멤버의 개발환경 구축 등  

패키지가 코드를 재사용하고 조건을 통일하는 수단인데 반해 컨테이너는 애플리케이션의 다양한 서비스를 만드는 수단임  
이들은 아키텍처에 대한 관심사의 분리(SoC)를 도와줌  
서비스는 애플리케이션의 나머지 부분과 독립적인 기능을 캡슐화하는 또 다른 종류의 컴포넌트임  
컨테이너는 유지보수가 가능한 형태로 디자인되어야 함  
책임을 명확하게 구분하여 해당 서비스의 변경으로 인해 애플리케이션의 다른 부분에 영향을 미치지 않도록 해야함  